In [1]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import xarray as xr

from theano import tensor as tt

print(f"Running on PyMC3 v{pm.__version__}")
warnings.simplefilter(action="ignore", category=FutureWarning)
RANDOM_SEED = 42
np.random.seed(42)

Running on PyMC3 v3.11.2


In [2]:
az.style.use("arviz-darkgrid")
df = pd.read_csv("data/UCAS.csv")

In [4]:
df = df.drop(columns=["Unnamed: 0"])

In [9]:
df.columns = df.columns.map(lambda x: x.replace(".", "_")).map(str.lower)

In [10]:
df.columns

Index(['year', 'simd_2012_2016_quintile', 'gender', 'age_group',
       'scottish_applicants', 'accepted_scottish_applicants'],
      dtype='object')

In [12]:
df["acceptance_rate"] = df["accepted_scottish_applicants"]/df["scottish_applicants"]

In [19]:
quintiles = df.simd_2012_2016_quintile.unique()
quintile_lookup = dict(zip(quintiles, range(len(quintiles))))
years = df.year.unique()
year_lookup = dict(zip(years, range(len(years))))
gender = df.gender.unique()
gender_lookup = dict(zip(gender, range(len(gender))))
age_group = df.age_group.unique()
age_lookup = dict(zip(age_group, range(len(age_group))))

In [26]:
quintile = df.simd_2012_2016_quintile = df.simd_2012_2016_quintile.replace(quintile_lookup).values
year = df.year = df.year.replace(year_lookup).values
gender = df.gender = df.gender.replace(gender_lookup).values
age = df.age_group = df.age_group.replace(age_lookup).values

In [28]:
acceptance_rate = df.acceptance_rate

## Pooled model

In [33]:
with pm.Model() as pooled:
    alpha = pm.Normal("alpha", 0.0, sigma=10.0)
    sigma = pm.Exponential("sigma", 1.0)
    y = pm.Normal("y", alpha, sigma=sigma, observed=acceptance_rate)

In [35]:
pm.model_to_graphviz(pooled)

ImportError: This function requires the python library graphviz, along with binaries. The easiest way to install all of this is by running

	conda install -c conda-forge python-graphviz